This notebook contains the PySAL/spreg code for Chapter 5 - OLS 

in
Modern Spatial Econometrics in Practice: A Guide to GeoDa, GeoDaSpace and PySAL.

by Luc Anselin and Sergio J. Rey

(c) 2014 Luc Anselin and Sergio J. Rey, All Rights Reserved

In [48]:
__author__ = "Luc Anselin luc.anselin@asu.edu"

## Basic Regression Setup##

**Creating arrays for y and x using the Baltimore example**

Preliminaries, importing **numpy** and **PySAL**

In [49]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [50]:
import numpy as np
import pysal

Loading the data set and creating the data object

This assumes that the sample data sets are contained in the 'data' subdirectory of the current working directory

In [51]:
db = pysal.open('data/baltim.dbf','r')

Quick check on the number of observations and the contents of the data set in the db object

In [52]:
len(db)

211

In [53]:
db.header

['STATION',
 'PRICE',
 'NROOM',
 'DWELL',
 'NBATH',
 'PATIO',
 'FIREPL',
 'AC',
 'BMENT',
 'NSTOR',
 'GAR',
 'AGE',
 'CITCOU',
 'LOTSZ',
 'SQFT',
 'X',
 'Y']

**y** - dependent variable is PRICE

In [54]:
y_name = "PRICE"

Create the y array as a n by 1 column vector (hence the transpose **T**)

In [55]:
y = np.array([db.by_col(y_name)]).T

Check on the dimensions

In [56]:
y.shape

(211, 1)

**x** - the explanatory variables

First create a list with the variable names, then use a list comprehension to create the **x** array

In [57]:
x_names = ['NROOM','NBATH','PATIO','FIREPL','AC','GAR','AGE',
           'LOTSZ','SQFT']
x = np.array([db.by_col(var) for var in x_names]).T

Check on dimensions

In [58]:
x.shape

(211, 9)

**Model weights** - needed for spatial diagnostics

k nearest neighbor with k=4 constructed from baltim.shp, using STATION as the ID variable

In [59]:
w = pysal.knnW_from_shapefile('data/baltim.shp',
                                k=4,idVariable='STATION')

Quick check on dimension

In [60]:
w.n

211

row-standardize - is **always** necessary

In [61]:
w.transform = 'r'

Quick check on the values of the weights

In [62]:
w.weights[1]

[0.25, 0.25, 0.25, 0.25]

**Kernel weights** - needed for HAC standard error option

triangular adaptive bandwidth kernel with k=12, constructed from baltim shape file

In [63]:
kw = pysal.adaptive_kernelW_from_shapefile('data/baltim.shp',
                                             k=12,diagonal=True,idVariable='STATION')

Check on dimension and actual weights - Note how diagonal element = 1, not 0

This is the case for a triangular kernel function, whether or not the **diagonal** option is set to True or not. Otherwise
the latter **must** be set to **True** (see also the example in Chapter 6).

In [64]:
kw.n

211

In [65]:
kw.weights[1]

[1.0,
 0.4849212978701576,
 0.3611234988877102,
 0.33567595079889523,
 0.31302206649524533,
 0.2768409033403173,
 0.27156871375844516,
 0.14137042299357871,
 0.1179222999771653,
 0.08526805755421585,
 0.0564397505258174,
 0.007682671413889897,
 9.99999900663795e-08]

##Basic OLS##

**Default settings, no variable names**

In [66]:
ols1 = pysal.spreg.OLS(y,x)

the regression coefficiens, in the order of the columns of **x**

In [67]:
ols1

<pysal.spreg.ols.OLS instance at 0x106dabe60>

In [68]:
dir(ols1)

['__doc__',
 '__init__',
 '__module__',
 '__summary',
 '_cache',
 'aic',
 'ar2',
 'betas',
 'breusch_pagan',
 'f_stat',
 'jarque_bera',
 'k',
 'koenker_bassett',
 'logll',
 'mean_y',
 'mulColli',
 'n',
 'name_ds',
 'name_gwk',
 'name_w',
 'name_x',
 'name_y',
 'predy',
 'r2',
 'robust',
 'schwarz',
 'sig2',
 'sig2ML',
 'sig2n',
 'sig2n_k',
 'std_err',
 'std_y',
 'summary',
 't_stat',
 'title',
 'u',
 'utu',
 'vm',
 'x',
 'xtx',
 'xtxi',
 'y']

In [69]:
ols1.betas

array([[ 23.26999634],
       [  0.22249352],
       [  5.64840507],
       [ 10.33587549],
       [ 11.17272765],
       [  7.85416373],
       [  5.40206849],
       [ -0.21345497],
       [  0.09490639],
       [  0.18775621]])

In [70]:
ols1.betas

array([[ 23.26999634],
       [  0.22249352],
       [  5.64840507],
       [ 10.33587549],
       [ 11.17272765],
       [  7.85416373],
       [  5.40206849],
       [ -0.21345497],
       [  0.09490639],
       [  0.18775621]])

pretty output, but no variable names (coefficients in order of columns of **x**) or any other descriptive information about 
the data set, etc.

In [71]:
print ols1.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Dependent Variable  :     dep_var                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

------------------------------------------------------------------------------------
            Variable     C

**OLS with variable and data set names**

In [72]:
ols1a = pysal.spreg.OLS(y,x,name_y=y_name,name_x=x_names,
                        name_ds='baltim.shp')

In [73]:
print ols1a.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

------------------------------------------------------------------------------------
            Variable     C

note how the order of the coefficients is different from before, it follows the alfabetical order of the
variable names

##OLS with White Test##

set **white_test = True**

In [74]:
ols2 = pysal.spreg.OLS(y,x,white_test=True,name_y=y_name,
                       name_x=x_names,name_ds='baltim.shp')

In [75]:
print ols2.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

------------------------------------------------------------------------------------
            Variable     C

now the **White** test is included as one of the diagnostics

##OLS with Spatial Diagnostics##

specify the weights (**w**) and set **spat_diag = True** and **moran = True**

specify a name for the weights in **name_w**, all the rest is as before

for convenience, white_test is back to default of False

In [76]:
ols3 = pysal.spreg.OLS(y,x,w=w,spat_diag=True,moran=True,
                       name_y=y_name,name_x=x_names,name_w='baltim_k4',
                       name_ds='baltim.shp')

In [77]:
print ols3.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Weights matrix      :   baltim_k4
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

-----------------------------------------------------------------------------

diagnostics for spatial dependence at the bottom of the listing

## OLS with White Standard Errors##

set **robust= 'white'** for the default scaling, dividing by $n - k$, the spatial diagnostics have been turned off

In [78]:
ols4 = pysal.spreg.OLS(y,x,robust='white',
                       name_y=y_name,name_x=x_names,name_ds='baltim.shp')

In [79]:
print ols4.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

White Standard Errors
------------------------------------------------------------------------------------
    

turn off scaling (i.e., divide by n) by setting **sig2n_k = False**

In [80]:
ols5 = pysal.spreg.OLS(y,x,robust='white',sig2n_k=False,
                       name_y=y_name,name_x=x_names,name_ds='baltim.shp')

In [81]:
print ols5.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     194.125                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      13.933                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

White Standard Errors
------------------------------------------------------------------------------------
    

note how all the estimates remain the same, but the standard errors change (slightly)

## OLS with HAC Standard Errors##

set **robust = 'hac'**, specify the kernel weights **gwk = kw** and give a name for the weights in **name_gwk**

In [82]:
ols6 = pysal.spreg.OLS(y,x,robust='hac',gwk=kw,
                       name_y=y_name,name_x=x_names,
                       name_gwk='baltim_tri_k12',name_ds='baltim.shp')

In [83]:
pysal.spreg.OLS()

TypeError: __init__() takes at least 3 arguments (1 given)

In [84]:
print ols6.summary

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  baltim.shp
Dependent Variable  :       PRICE                Number of Observations:         211
Mean dependent var  :     44.3072                Number of Variables   :          10
S.D. dependent var  :     23.6061                Degrees of Freedom    :         201
R-squared           :      0.6500
Adjusted R-squared  :      0.6343
Sum squared residual:   40960.463                F-statistic           :     41.4718
Sigma-square        :     203.783                Prob(F-statistic)     :    3.24e-41
S.E. of regression  :      14.275                Log likelihood        :    -855.223
Sigma-square ML     :     194.125                Akaike info criterion :    1730.446
S.E of regression ML:     13.9329                Schwarz criterion     :    1763.965

HAC Standard Errors; Kernel Weights: baltim_tri_k12
-----------------------------------------------------------

Again, same estimates as before, but different standard errors

## Accessing Individual Functions##

**Muticollinearity Diagnostics** - Variance inflation factor

use **vif** and pass a regression object, e.g., **ols1**

In [85]:
v = pysal.spreg.diagnostics.vif(ols1)

In [86]:
v

[None,
 (2.1282653585417042, 0.46986622038767001),
 (1.7612737468067754, 0.56777091114485756),
 (1.2635001075481409, 0.79145224763021937),
 (1.4167716982292522, 0.70583002275514606),
 (1.3859551858036305, 0.72152405088059268),
 (1.4320753736980556, 0.69828726781167594),
 (1.3336259007911946, 0.74983546690772451),
 (1.5646798143926104, 0.63910839189050817),
 (2.4833041895651888, 0.40268928961743256)]

extract the statistics by skipping over the first element in the list, **v[1:]**

In [87]:
v[1:]

[(2.1282653585417042, 0.46986622038767001),
 (1.7612737468067754, 0.56777091114485756),
 (1.2635001075481409, 0.79145224763021937),
 (1.4167716982292522, 0.70583002275514606),
 (1.3859551858036305, 0.72152405088059268),
 (1.4320753736980556, 0.69828726781167594),
 (1.3336259007911946, 0.74983546690772451),
 (1.5646798143926104, 0.63910839189050817),
 (2.4833041895651888, 0.40268928961743256)]

This list contains a tuple with for each variable (in the order given in **x**) the variance inflation factor (VIF) and its inverse, the tolerance factor. The rule of thumb is to keep the VIF under 5 or 10 and the tolerance factor above 0.2 or 0.1.

**Lagrange Multiplier Tests** - accessing individual statistics

use **LMtests** with a regression object and a weights object

In [88]:
vw = pysal.spreg.diagnostics_sp.LMtests(ols1,w)

In [89]:
vw

<pysal.spreg.diagnostics_sp.LMtests instance at 0x1076f8b00>

check the contents of this object using **dir**

In [90]:
dir(vw)

['__doc__', '__init__', '__module__', 'lme', 'lml', 'rlme', 'rlml', 'sarma']

extract the specific test statistic as attributes, e.g., **lme, lml,
rlme, rlml, sarma**

In [91]:
vw.lme

(5.2825246195978197, 0.021540492505405306)

In [92]:
vw.lml

(31.407013776695475, 2.0922374390272532e-08)

## Practice##

Create a regression object using the classic Harrison-Rubinfeld Boston house price data set (included as Boston.shp). Regression median house value (MEDV) on crime rate (CRIM), Charles river dummy (CHAS), nitric oxides (NOX), number of rooms (RM), age (AGE), weighted distance to five employment centers (DIS) and percent "lower status population" (LSTAT). A full description of the Boston data set is available on the GeoDa Center sample data set site.

Use a k-nearest neighbor spatial weights (k = 4) for the spatial diagnostics, and a triangular adaptive bandwidth kernel weights (k = 12) for the HAC standard errors.

What is the most likely alternative spatial regression model, given the results of a spatial specification search.

Try including the White test and see what happens.

Try any other combinations of explanatory variables, spatial weights and standard error specifications.